### ingesting secondary dataset ###

We are including 1 table from a potential secondary dataset to demonstrate a few outer joins. The table we are using to facilitate this demonstration reports education levels for different counties across the United States.

Since some jurisdictions appear under multiple names but the same underlying fipscode, we currently have the relationship between education and Jurisdiction as 1:m.

In [70]:
dataset_id = "education"

1. We compare the total votes between the 2016 and 2018 elections in the House for Texas jurisdictions. This INNER JOIN query can be used to compare voter turnout. 

In [19]:
%%bigquery 
SELECT DISTINCT upper(e1.jname) as jname,  e1.year, e1.office, e1.total_votes
FROM hdv_modeled.Results e2
JOIN hdv_modeled.Results e1
ON e1.year = e2.year
WHERE e1.state = 'TX' and e1.office = 'US House'
ORDER by jname, year
limit 11

,jname,year,office,total_votes
0,ANDERSON,2016,US House,15341
1,ANDERSON,2018,US House,14603
2,ANDREWS,2016,US House,4528
3,ANDREWS,2018,US House,4123
4,ANGELINA,2016,US House,29539
5,ANGELINA,2018,US House,26257
6,ARANSAS,2016,US House,10382
7,ARANSAS,2018,US House,8917
8,ARCHER,2016,US House,4082
9,ARCHER,2018,US House,3603


2. We compare the total votes for the candidates running in the 2016 and 2018 House elections in various jurisdictions.

In [30]:
%%bigquery 
SELECT DISTINCT hs1.cname, hs1.fipscode, hs1.votes, upper(hs1.jname) as jname, hs1.party, hs1.year, hs1.office 
FROM hdv_modeled.Results hs1
JOIN hdv_modeled.Results hs2
ON hs1.cname = hs2.cname and hs1.year = hs2.year  
WHERE hs1.jname = hs2.jname and hs1.office = 'US House'
ORDER by jname, hs1.cname, hs1.year
limit 11

,cname,fipscode,votes,jname,party,year,office
0,Dave Moore,4500100000,83,ABBEVILLE,American,2018,US House
1,Hosea Cleveland,4500100000,3487,ABBEVILLE,Democrat,2016,US House
2,Jeff Duncan,4500100000,6859,ABBEVILLE,Republican,2016,US House
3,Jeff Duncan,4500100000,5544,ABBEVILLE,Republican,2018,US House
4,Mary Geren,4500100000,2943,ABBEVILLE,Democrat,2018,US House
5,Write-In,4500100000,6,ABBEVILLE,None,2016,US House
6,Write-In,4500100000,6,ABBEVILLE,unknown,2018,US House
7,BLANK VOTES,2302100100,23,ABBOT,Independent,2016,US House
8,"Bond, Tiffany L.",2302100100,20,ABBOT,Independent,2018,US House
9,"Cain, Emily Ann",2302100100,123,ABBOT,Democrat,2016,US House


3. We compare the total votes given in the House election as opposed to the Presidential election. We expect these values to be very close, but for some reason, people may choose to abstain from 1 election even though both are on the same ballot. This query can give us some information on voters who may vote on some principles or on a matter of conscientiousness.

In [34]:
%%bigquery 
SELECT DISTINCT hs1.jname, hs1.fipscode, hs1.total_votes, hs1.office, hs1.year
FROM hdv_modeled.Results hs1
JOIN hdv_modeled.Jurisdiction hs2
ON hs1.fipscode = hs2.fipscode
WHERE hs1.state = 'TX' and (hs1.office = 'US House' or hs1.office = 'President') and hs1.year = 2016
ORDER by hs1.jname
limit 11

,jname,fipscode,total_votes,office,year
0,ANDERSON,4800100000,15341,US House,2016
1,ANDERSON,4800100000,16977,President,2016
2,ANDREWS,4800300000,4528,US House,2016
3,ANDREWS,4800300000,4943,President,2016
4,ANGELINA,4800500000,29539,US House,2016
5,ANGELINA,4800500000,29911,President,2016
6,ARANSAS,4800700000,10382,US House,2016
7,ARANSAS,4800700000,10512,President,2016
8,ARCHER,4800900000,4082,US House,2016
9,ARCHER,4800900000,4283,President,2016


4. We compare Senate 2016 and Presidential 2016 total vote counts across different jurisdictions. This can tell us similar information as the previous query. This specific query is for Alaska.


In [44]:
%%bigquery 
SELECT DISTINCT hs1.jname, hs1.fipscode, hs1.total_votes, hs1.office, hs1.year
FROM hdv_modeled.Results hs1
JOIN hdv_modeled.Jurisdiction hs2
ON hs1.fipscode = hs2.fipscode
WHERE hs1.state = 'AK' and (hs1.office = 'Senate' or hs1.office = 'President') and hs1.year = 2016
ORDER by hs1.jname
limit 11

,jname,fipscode,total_votes,office,year
0,District 1,290100000,6413,Senate,2016
1,District 1,290100000,6638,President,2016
2,District 10,291000000,8837,Senate,2016
3,District 10,291000000,9040,President,2016
4,District 11,291100000,9546,Senate,2016
5,District 11,291100000,9689,President,2016
6,District 12,291200000,9394,Senate,2016
7,District 12,291200000,9543,President,2016
8,District 13,291300000,6190,Senate,2016
9,District 13,291300000,6533,President,2016


5. Left Outer Join comparing total House 2016 votes against the number of college graduates across various jurisdictions. Here, we can relate how education may impact voter turnout, especially for a particular population segment.

In [41]:
%%bigquery 
SELECT DISTINCT s1.fipscode, s1.total_votes, s1.jname, s1.state, s2.College, s2.FIPS_Code
FROM hdv_modeled.Results s1
LEFT OUTER JOIN education.education s2
ON (s1.fipscode/100000) = s2.FIPS_Code
WHERE not (s1.state = 'AK')
limit 11

,fipscode,total_votes,jname,state,College,FIPS_Code
0,100100000,24973,Autauga,AL,10291,1001
1,100300000,95215,Baldwin,AL,46075,1003
2,100500000,10469,Barbour,AL,2220,1005
3,100700000,8819,Bibb,AL,1813,1007
4,100900000,25588,Blount,AL,5010,1009
5,101100000,4710,Bullock,AL,945,1011
6,101300000,8732,Butler,AL,2235,1013
7,101500000,47864,Calhoun,AL,14219,1015
8,101700000,13900,Chambers,AL,3118,1017
9,101900000,10733,Cherokee,AL,2407,1019


6. Right Outer Join comparing the number of high school graduates without college experience against the total House 2016 election votes in various jurisdictions. Overall, this is a similar query to 5, but on the other side of the education spectrum.

In [43]:
%%bigquery 
SELECT DISTINCT s1.fipscode, s1.total_votes, s1.jname, s1.state, s2.__High_School, s2.FIPS_Code
FROM hdv_modeled.Results s1
LEFT OUTER JOIN education.education s2
ON (s1.fipscode/100000) = s2.FIPS_Code
WHERE (s1.state = 'TX')
limit 11

,fipscode,total_votes,jname,state,__High_School,FIPS_Code
0,4800100000,16977,ANDERSON,TX,8351,48001
1,4800300000,4943,ANDREWS,TX,2695,48003
2,4800500000,29911,ANGELINA,TX,10826,48005
3,4800700000,10512,ARANSAS,TX,2742,48007
4,4800900000,4283,ARCHER,TX,558,48009
5,4801100000,1021,ARMSTRONG,TX,164,48011
6,4801300000,13673,ATASCOSA,TX,7032,48013
7,4801500000,12274,AUSTIN,TX,3402,48015
8,4801700000,1793,BAILEY,TX,1142,48017
9,4801900000,10218,BANDERA,TX,1781,48019
